In [1]:
# Environment Variables: 

YES = 1
NO = 0

In [ ]:
# Imports for Training
check_GPU = NO
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import qkeras
from qkeras.estimate import print_qstats
from qkeras.utils import model_quantize
from qkeras.utils import quantized_model_dump

%matplotlib inline

In [13]:
# Custom Loss Function (TENSORFLOW):
def custom_weighted_mse_loss(I, J, n):
    W = tf.pow(I, n)

    squared_diffs = tf.pow(I - J, 2)

    weighted_squared_diffs = W * squared_diffs

    loss = tf.reduce_mean(weighted_squared_diffs)

    return loss

In [14]:
# Load Unquantized Model
with tf.keras.utils.custom_object_scope({'custom_weighted_mse_loss': custom_weighted_mse_loss}):
    model = tf.keras.models.load_model("Gaussian_Model.keras")

In [15]:
# Post Training Quantization
save_qmodel = YES
view_qstats = NO
fractional = 4
integer = 0
symmetric = YES
total_bits = fractional + integer
Gaussian_Model_dir = '/mnt/Research/Data/Models/'
Quantized_Gaussian_Model_file = Gaussian_Model_dir + f'Gaussian_Model_{total_bits}_{integer}.keras'

q_dict = {
    "QConv2D": {
        "kernel_quantizer": f"quantized_bits({total_bits},{integer},{symmetric})",
        "bias_quantizer": f"quantized_bits({total_bits},{integer},{symmetric})"
    },
    "QDense": {
        "kernel_quantizer": f"quantized_bits({total_bits},{integer},{symmetric})",
        "bias_quantizer": f"quantized_bits({total_bits},{integer},{symmetric})"
    },
    "QBatchNormalization": {},

    "QActivation": f"quantized_relu({total_bits},{integer})"
}

qmodel = model_quantize(model, q_dict, total_bits, transfer_weights=True)


if save_qmodel:
    qmodel.save(Quantized_Gaussian_Model_file)

if view_qstats:
    print_qstats(qmodel)